In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel("DEBUG")

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
locs += np.array([0,0,-0.70])
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    print(q)
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/package.xml' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/pinball_iiwas_prm_certify.runfiles/drake/package.xml' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/pinball_iiwas_prm_certify.runfiles/drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is n

[-1.7, -1.7, 0]
[1.7, 1.7, 0]


In [6]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

In [8]:
visualizer.meshcat2.Delete("/prm")

In [18]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)


num_points = 10 # 5000

PRM = prm.PRM( 
            limits_t,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

PRM_bad = prm.PRM( 
            limits_t,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle_bad,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

# PRM.add_start_end(start, target)
if num_points < 100:
    PRM.plot()
    PRM_bad.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

[PRM] Samples 0
[PRM] Nodes connected: 0
[PRM] Samples 0
[PRM] Nodes connected: 0


## Set up the sliders so we can move the plant around manually

In [19]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [20]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


In [21]:
# t0 = time.time()
# line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
#                                 rational_forward_kinematics.SeparatingPlaneOrder.kAffine, np.zeros(2),
#                                 set(), VerificationOption()
#                                )
# t1 = time.time()
# print(f"Time to construct line certifier = {t1-t0}s")

In [22]:
def make_prm_line_pairs(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    s0 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    s1 = np.zeros((len(endpoint_index_set), plant.num_positions()))
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0[i] = PRM.nodes[idx0]
        s1[i] = PRM.nodes[idx1]
    return s0, s1
s0_mat_safe, s1_mat_safe = make_prm_line_pairs(PRM)
s0_mat_unsafe, s1_mat_unsafe = make_prm_line_pairs(PRM_bad)
print(s0_mat_safe.shape)
print(s0_mat_unsafe.shape)

(27, 2)
(6, 2)


In [23]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe

In [24]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe
def make_certifier(use_affine = True, num_points = -1):
    t0 = time.time()
    plane_type = rational_forward_kinematics.SeparatingPlaneOrder.kAffine if use_affine \
                    else rational_forward_kinematics.SeparatingPlaneOrder.kConstant
    line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                    plane_type, 
                                    q_star[:2],
                                    set(), VerificationOption()
                                   )
    t1 = time.time()
    print(f"Time to construct line certifier = {t1-t0}s")
    return line_certifier

def do_certify(line_certifier, num_points = -1):
    t0 = time.time()
    safe_points, planes = line_certifier.CertifyTangentConfigurationSpaceLines(s0_mat_safe[:num_points,:], s1_mat_safe[:num_points,:])
    t1 = time.time()
    print(f"Certification of safe PRM in {t1-t0}s")
    num_safe1 = count_num_safe(safe_points)
    print(f"Certified {num_safe1}/{len(safe_points)} as safe")
    print(f"Failed to certify {len(safe_points)-num_safe1}/{len(safe_points)} safe points as safe")
    
    return (safe_points, planes, num_safe1)

In [25]:
line_certifier = make_certifier(use_affine = True)

DEBUG:drake:Done rational 1/2080
DEBUG:drake:Done rational 2/2080
DEBUG:drake:Done rational 3/2080
DEBUG:drake:Done rational 4/2080
DEBUG:drake:Done rational 5/2080
DEBUG:drake:Done rational 6/2080
DEBUG:drake:Done rational 7/2080
DEBUG:drake:Done rational 8/2080
DEBUG:drake:Done rational 9/2080
DEBUG:drake:Done rational 10/2080
DEBUG:drake:Done rational 11/2080
DEBUG:drake:Done rational 12/2080
DEBUG:drake:Done rational 13/2080
DEBUG:drake:Done rational 14/2080
DEBUG:drake:Done rational 15/2080
DEBUG:drake:Done rational 16/2080
DEBUG:drake:Done rational 17/2080
DEBUG:drake:Done rational 18/2080
DEBUG:drake:Done rational 19/2080
DEBUG:drake:Done rational 20/2080
DEBUG:drake:Done rational 21/2080
DEBUG:drake:Done rational 22/2080
DEBUG:drake:Done rational 23/2080
DEBUG:drake:Done rational 24/2080
DEBUG:drake:Done rational 25/2080
DEBUG:drake:Done rational 26/2080
DEBUG:drake:Done rational 27/2080
DEBUG:drake:Done rational 28/2080
DEBUG:drake:Done rational 29/2080
DEBUG:drake:Done ration

DEBUG:drake:Done rational 239/2080
DEBUG:drake:Done rational 240/2080
DEBUG:drake:Done rational 241/2080
DEBUG:drake:Done rational 242/2080
DEBUG:drake:Done rational 243/2080
DEBUG:drake:Done rational 244/2080
DEBUG:drake:Done rational 245/2080
DEBUG:drake:Done rational 246/2080
DEBUG:drake:Done rational 247/2080
DEBUG:drake:Done rational 248/2080
DEBUG:drake:Done rational 249/2080
DEBUG:drake:Done rational 250/2080
DEBUG:drake:Done rational 251/2080
DEBUG:drake:Done rational 252/2080
DEBUG:drake:Done rational 253/2080
DEBUG:drake:Done rational 254/2080
DEBUG:drake:Done rational 255/2080
DEBUG:drake:Done rational 256/2080
DEBUG:drake:Done rational 257/2080
DEBUG:drake:Done rational 258/2080
DEBUG:drake:Done rational 259/2080
DEBUG:drake:Done rational 260/2080
DEBUG:drake:Done rational 261/2080
DEBUG:drake:Done rational 262/2080
DEBUG:drake:Done rational 263/2080
DEBUG:drake:Done rational 264/2080
DEBUG:drake:Done rational 265/2080
DEBUG:drake:Done rational 266/2080
DEBUG:drake:Done rat

DEBUG:drake:Done rational 474/2080
DEBUG:drake:Done rational 475/2080
DEBUG:drake:Done rational 476/2080
DEBUG:drake:Done rational 477/2080
DEBUG:drake:Done rational 478/2080
DEBUG:drake:Done rational 479/2080
DEBUG:drake:Done rational 480/2080
DEBUG:drake:Done rational 481/2080
DEBUG:drake:Done rational 482/2080
DEBUG:drake:Done rational 483/2080
DEBUG:drake:Done rational 484/2080
DEBUG:drake:Done rational 485/2080
DEBUG:drake:Done rational 486/2080
DEBUG:drake:Done rational 487/2080
DEBUG:drake:Done rational 488/2080
DEBUG:drake:Done rational 489/2080
DEBUG:drake:Done rational 490/2080
DEBUG:drake:Done rational 491/2080
DEBUG:drake:Done rational 492/2080
DEBUG:drake:Done rational 493/2080
DEBUG:drake:Done rational 494/2080
DEBUG:drake:Done rational 495/2080
DEBUG:drake:Done rational 496/2080
DEBUG:drake:Done rational 497/2080
DEBUG:drake:Done rational 498/2080
DEBUG:drake:Done rational 499/2080
DEBUG:drake:Done rational 500/2080
DEBUG:drake:Done rational 501/2080
DEBUG:drake:Done rat

DEBUG:drake:Done rational 709/2080
DEBUG:drake:Done rational 710/2080
DEBUG:drake:Done rational 711/2080
DEBUG:drake:Done rational 712/2080
DEBUG:drake:Done rational 713/2080
DEBUG:drake:Done rational 714/2080
DEBUG:drake:Done rational 715/2080
DEBUG:drake:Done rational 716/2080
DEBUG:drake:Done rational 717/2080
DEBUG:drake:Done rational 718/2080
DEBUG:drake:Done rational 719/2080
DEBUG:drake:Done rational 720/2080
DEBUG:drake:Done rational 721/2080
DEBUG:drake:Done rational 722/2080
DEBUG:drake:Done rational 723/2080
DEBUG:drake:Done rational 724/2080
DEBUG:drake:Done rational 725/2080
DEBUG:drake:Done rational 726/2080
DEBUG:drake:Done rational 727/2080
DEBUG:drake:Done rational 728/2080
DEBUG:drake:Done rational 729/2080
DEBUG:drake:Done rational 730/2080
DEBUG:drake:Done rational 731/2080
DEBUG:drake:Done rational 732/2080
DEBUG:drake:Done rational 733/2080
DEBUG:drake:Done rational 734/2080
DEBUG:drake:Done rational 735/2080
DEBUG:drake:Done rational 736/2080
DEBUG:drake:Done rat

DEBUG:drake:Done rational 944/2080
DEBUG:drake:Done rational 945/2080
DEBUG:drake:Done rational 946/2080
DEBUG:drake:Done rational 947/2080
DEBUG:drake:Done rational 948/2080
DEBUG:drake:Done rational 949/2080
DEBUG:drake:Done rational 950/2080
DEBUG:drake:Done rational 951/2080
DEBUG:drake:Done rational 952/2080
DEBUG:drake:Done rational 953/2080
DEBUG:drake:Done rational 954/2080
DEBUG:drake:Done rational 955/2080
DEBUG:drake:Done rational 956/2080
DEBUG:drake:Done rational 957/2080
DEBUG:drake:Done rational 958/2080
DEBUG:drake:Done rational 959/2080
DEBUG:drake:Done rational 960/2080
DEBUG:drake:Done rational 961/2080
DEBUG:drake:Done rational 962/2080
DEBUG:drake:Done rational 963/2080
DEBUG:drake:Done rational 964/2080
DEBUG:drake:Done rational 965/2080
DEBUG:drake:Done rational 966/2080
DEBUG:drake:Done rational 967/2080
DEBUG:drake:Done rational 968/2080
DEBUG:drake:Done rational 969/2080
DEBUG:drake:Done rational 970/2080
DEBUG:drake:Done rational 971/2080
DEBUG:drake:Done rat

...

DEBUG:drake:constructing tuple 1433/2080
DEBUG:drake:constructing tuple 1434/2080
DEBUG:drake:constructing tuple 1435/2080
DEBUG:drake:constructing tuple 1436/2080
DEBUG:drake:constructing tuple 1437/2080
DEBUG:drake:constructing tuple 1438/2080
DEBUG:drake:constructing tuple 1439/2080
DEBUG:drake:constructing tuple 1440/2080
DEBUG:drake:constructing tuple 1441/2080
DEBUG:drake:constructing tuple 1442/2080
DEBUG:drake:constructing tuple 1443/2080
DEBUG:drake:constructing tuple 1444/2080
DEBUG:drake:constructing tuple 1445/2080
DEBUG:drake:constructing tuple 1446/2080
DEBUG:drake:constructing tuple 1447/2080
DEBUG:drake:constructing tuple 1448/2080
DEBUG:drake:constructing tuple 1449/2080
DEBUG:drake:constructing tuple 1450/2080
DEBUG:drake:constructing tuple 1451/2080
DEBUG:drake:constructing tuple 1452/2080
DEBUG:drake:constructing tuple 1453/2080
DEBUG:drake:constructing tuple 1454/2080
DEBUG:drake:constructing tuple 1455/2080
DEBUG:drake:constructing tuple 1456/2080
DEBUG:drake:cons

DEBUG:drake:constructing tuple 1633/2080
DEBUG:drake:constructing tuple 1634/2080
DEBUG:drake:constructing tuple 1635/2080
DEBUG:drake:constructing tuple 1636/2080
DEBUG:drake:constructing tuple 1637/2080
DEBUG:drake:constructing tuple 1638/2080
DEBUG:drake:constructing tuple 1639/2080
DEBUG:drake:constructing tuple 1640/2080
DEBUG:drake:constructing tuple 1641/2080
DEBUG:drake:constructing tuple 1642/2080
DEBUG:drake:constructing tuple 1643/2080
DEBUG:drake:constructing tuple 1644/2080
DEBUG:drake:constructing tuple 1645/2080
DEBUG:drake:constructing tuple 1646/2080
DEBUG:drake:constructing tuple 1647/2080
DEBUG:drake:constructing tuple 1648/2080
DEBUG:drake:constructing tuple 1649/2080
DEBUG:drake:constructing tuple 1650/2080
DEBUG:drake:constructing tuple 1651/2080
DEBUG:drake:constructing tuple 1652/2080
DEBUG:drake:constructing tuple 1653/2080
DEBUG:drake:constructing tuple 1654/2080
DEBUG:drake:constructing tuple 1655/2080
DEBUG:drake:constructing tuple 1656/2080
DEBUG:drake:cons

DEBUG:drake:constructing tuple 1833/2080
DEBUG:drake:constructing tuple 1834/2080
DEBUG:drake:constructing tuple 1835/2080
DEBUG:drake:constructing tuple 1836/2080
DEBUG:drake:constructing tuple 1837/2080
DEBUG:drake:constructing tuple 1838/2080
DEBUG:drake:constructing tuple 1839/2080
DEBUG:drake:constructing tuple 1840/2080
DEBUG:drake:constructing tuple 1841/2080
DEBUG:drake:constructing tuple 1842/2080
DEBUG:drake:constructing tuple 1843/2080
DEBUG:drake:constructing tuple 1844/2080
DEBUG:drake:constructing tuple 1845/2080
DEBUG:drake:constructing tuple 1846/2080
DEBUG:drake:constructing tuple 1847/2080
DEBUG:drake:constructing tuple 1848/2080
DEBUG:drake:constructing tuple 1849/2080
DEBUG:drake:constructing tuple 1850/2080
DEBUG:drake:constructing tuple 1851/2080
DEBUG:drake:constructing tuple 1852/2080
DEBUG:drake:constructing tuple 1853/2080
DEBUG:drake:constructing tuple 1854/2080
DEBUG:drake:constructing tuple 1855/2080
DEBUG:drake:constructing tuple 1856/2080
DEBUG:drake:cons

DEBUG:drake:constructing tuple 2033/2080
DEBUG:drake:constructing tuple 2034/2080
DEBUG:drake:constructing tuple 2035/2080
DEBUG:drake:constructing tuple 2036/2080
DEBUG:drake:constructing tuple 2037/2080
DEBUG:drake:constructing tuple 2038/2080
DEBUG:drake:constructing tuple 2039/2080
DEBUG:drake:constructing tuple 2040/2080
DEBUG:drake:constructing tuple 2041/2080
DEBUG:drake:constructing tuple 2042/2080
DEBUG:drake:constructing tuple 2043/2080
DEBUG:drake:constructing tuple 2044/2080
DEBUG:drake:constructing tuple 2045/2080
DEBUG:drake:constructing tuple 2046/2080
DEBUG:drake:constructing tuple 2047/2080
DEBUG:drake:constructing tuple 2048/2080
DEBUG:drake:constructing tuple 2049/2080
DEBUG:drake:constructing tuple 2050/2080
DEBUG:drake:constructing tuple 2051/2080
DEBUG:drake:constructing tuple 2052/2080
DEBUG:drake:constructing tuple 2053/2080
DEBUG:drake:constructing tuple 2054/2080
DEBUG:drake:constructing tuple 2055/2080
DEBUG:drake:constructing tuple 2056/2080
DEBUG:drake:cons

Time to construct line certifier = 8.193992376327515s


In [26]:
ret_safe_affine, ret_unsafe_affine, num_safe = do_certify(line_certifier, num_points = 2)

RuntimeError: argument index out of range

In [ ]:
s1_mat_safe

In [ ]:
[:2,]